# 0 - Importando Bibliotecas

In [7]:
#!pip3 install geocoder

In [4]:
import pandas as pd
import geocoder
import glob

# 1 - Obtenção dos dados por geolocalização reversa

## Definindo o PATH dos arquivos

In [2]:
# Arquivos obtidos de fontes externas
file_toronto_listings = "../2-data/input/Toronto_2021-02-08/listings.csv"

# Onde salvar os arquivos com dados gerados, para utilização no script principal
processed_files = '../2-data/generated/'


## Importante: Utilizar a chave da API do Mapquest (OBS: Max 15k solicitações/mês, no plano gratuito)

In [9]:
#https://developer.mapquest.com/user/356679/app-detail/ca8799c9-2cf4-407c-8cd5-f7b92e1ebc47?from=1619302200&to=1619388600&metric=total_response_time&unit=hour#performance
#geocoderApiKey='oNsxug1crAZaJ2IEyVBC9Gcx1TKw81xm'
geocoderApiKey='kblRmFMxOIFWmNn7m4NBcjVA4mc9Zxw8'

In [ ]:
# Lendo o banco de dados principal, com os arquivos de toronto
df = pd.read_csv(file_toronto_listings, index_col = 'id')

In [10]:
# Testando a API, confirmando que funciona com exemplo individual
# location = geocoder.mapquest([df['latitude'].iloc[1], df['longitude'].iloc[x]], method='reverse', key=geocoderApiKey)
# location.raw

In [11]:
# Vamos usar as colunas 'latitude' e 'longitude' pra fazer uma busca reversa no Mapquest
# O Mapquest retorna diversas informações para cada par de lat-lon, mas estamos interess
# ados apenas na cidade, para esse estudo. 

# ~ 5 minutos p/ 1000 queries

# Criando uma df em branco
location_df = pd.DataFrame()

location = geocoder.mapquest([45.15, -75.14], method='reverse', key=geocoderApiKey)

#for x in range(1000):
#for x in range(1000, 2000):
#for x in range(2000, 3000):
#for x in range(3000, 4000):
#for x in range(4000, 5000):
#for x in range(5000, 6000):
#for x in range(6000, 7000):
#for x in range(7000, 8000):
#for x in range(8000, 9000):
#for x in range(9000, 10000):
#for x in range(10000, 11000):
#for x in range(11000, 12000):
#for x in range(12000, 13000):
#for x in range(13000, 14000):
#for x in range(14000, 15000):
for x in range(15000, 15832):
    location = geocoder.mapquest([df['latitude'].iloc[x], df['longitude'].iloc[x]], method='reverse', key=geocoderApiKey)
    
    location.raw['listing_id'] = str(df.index[x]) ##Adicionando o listing id para podermos buscar depois
    
    location.raw.pop('street', None)
    location.raw.pop('adminArea1Type', None)
    location.raw.pop('adminArea1', None)
    location.raw.pop('adminArea3Type', None)
    location.raw.pop('adminArea3', None)
    location.raw.pop('postalCode', None)
    location.raw.pop('geocodeQualityCode', None)
    location.raw.pop('geocodeQuality', None)
    location.raw.pop('dragPoint', None)
    location.raw.pop('sideOfStreet', None)
    location.raw.pop('linkId', None)
    location.raw.pop('unknownInput', None)
    location.raw.pop('type', None)
    location.raw.pop('latLng', None)
    location.raw.pop('displayLatLng', None)
    location.raw.pop('mapUrl', None)

    location.raw['Neighborhood'] = location.raw.pop('adminArea6')
    location.raw['City'] = location.raw.pop('adminArea5')
    location.raw['County'] = location.raw.pop('adminArea4')

    location.raw.pop('adminArea6Type', None)
    location.raw.pop('adminArea5Type', None)
    location.raw.pop('adminArea4Type', None)
    
    location_df = location_df.append(location.raw, ignore_index=True)
      
location_df = location_df.set_index('listing_id')

print(location_df.shape)
location_df.head()

(832, 3)


,City,County,Neighborhood
listing_id,,,
46970103,North York,Toronto,
46970784,Toronto,Toronto,
46970923,North York,Toronto,
46973442,East York,Toronto,
46984433,Toronto,Toronto,


In [13]:
#location_df.to_csv(processed_files+'geolocation/geolocation_data1000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data2000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data3000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data4000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data5000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data6000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data7000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data8000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data9000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data10000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data11000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data12000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data13000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data14000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data15000.csv')
#location_df.to_csv(processed_files+'geolocation/geolocation_data15832.csv')
location_df.to_csv(processed_files+'geolocation/geolocation_data15832.csv')


## Encontrando e unindo todos os .csv de geolocalização

In [5]:
# Encontrando todos os arquivos csv com dados de geolocalização
# Juntando todos eles em uma df única, e depois, exportando par
# a um csv completo

all_files = glob.glob(processed_files+'geolocation' + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.shape

(15832, 4)

## Gerando o arquivo final, com dados de cidade para todas as propriedades de Toronto

In [6]:
frame = frame.set_index('listing_id')
frame.to_csv(processed_files+'geolocation_data.csv')
